In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# ============================================================
# ✅ FIX PATH FOR NOTEBOOK INSIDE /notebooks
# ============================================================
import sys, os
from pathlib import Path

# Xác định thư mục cha của notebooks/
project_root = Path(os.getcwd()).parent  # => C:\Users\User\RR_model
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))  # ⚠️ thêm project_root, KHÔNG thêm src

print("📁 Project root added:", project_root)

# Kiểm tra lại
import importlib.util
print("🔍 src module found:", importlib.util.find_spec("src"))

📁 Project root added: C:\Users\MAFC4709\Python_work\RR_model
🔍 src module found: ModuleSpec(name='src', loader=<_frozen_importlib_external.SourceFileLoader object at 0x0000022AE04EDB40>, origin='C:\\Users\\MAFC4709\\Python_work\\RR_model\\src\\__init__.py', submodule_search_locations=['C:\\Users\\MAFC4709\\Python_work\\RR_model\\src'])


In [3]:
import pandas as pd
import numpy as np

from src.rollrate.transition import compute_transition_by_mob
from src.rollrate.forecast import forecast_all_vintages
from src.rollrate.forecast_plan import forecast_sale_plan_by_mob

# from src.rollrate.calibration import (
#     compute_k_per_product,
#     apply_calibration_to_forecast,
#     apply_calibration_to_sale_plan,
# )

from src.rollrate.seasonality import (
    build_seasonality,
    apply_seasonality_to_lifecycle,
    apply_seasonality_to_sale_plan
)

from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    combine_all_lifecycle_amount,
    lifecycle_to_long_df_amount,
    tag_forecast_rows_amount,
    add_del_metrics,
    aggregate_to_product,
    aggregate_products_to_portfolio,
    extend_actual_info_with_portfolio,
    export_lifecycle_all_products_one_file,
)

from src.config import CFG, BUCKETS_CANON


In [4]:
# ===========================================================
# 📘 Roll Rate Demo v7 - Sử dụng dữ liệu thật từ Oracle hoặc SQL file
# ===========================================================

from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))

from src.data_loader import load_data
from src.config import CFG, OUT_ROOT
from src.rollrate.transition import compute_transition_from_pairs, compute_transition_by_mob

from src.rollrate.export_excel import export_transition_excel
from src.rollrate.backtest import (
    compute_actual_matrix,
    rollforward_validation,
    plot_matrix_diff,
    plot_distribution_compare
)

print("✅ Import modules thành công.")
print("Cấu hình CFG:", CFG)


# --- 2️⃣ Load dữ liệu thực tế ---
sql_path = r"D:\Python_code\RR_Model_1711\sql\uns2.sql"

df = load_data(sql_path)
#df = load_data()
df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].apply(lambda x: x.replace(day=1))

#print(f"📦 Đã load {len(df):,} dòng dữ liệu từ {sql_path}")

# Kiểm tra các cột quan trọng
required_cols = [CFG["loan"], CFG["mob"], CFG["state"], CFG["ead"], CFG["cutoff"]]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"❌ Thiếu các cột cần thiết trong data: {missing}")

print("✅ Dữ liệu có đủ các cột cần thiết:", required_cols)

# Nếu chưa có PRODUCT_TYPE thì thêm mặc định
if "PRODUCT_TYPE" not in df.columns:
    df["PRODUCT_TYPE"] = "A"
    print("⚠️ Không có PRODUCT_TYPE trong data, gán mặc định = 'A'")

# Convert cutoff về datetime
df[CFG["cutoff"]] = pd.to_datetime(df[CFG["cutoff"]])





✅ Import modules thành công.
Cấu hình CFG: {'loan': 'AGREEMENT_ID', 'mob': 'MOB', 'state': 'STATE_MODEL', 'orig_date': 'DISBURSAL_DATE', 'ead': 'PRINCIPLE_OUTSTANDING', 'disb': 'DISBURSAL_AMOUNT', 'cutoff': 'CUTOFF_DATE'}
🔗 Loading data from Oracle...
=== SQL DEBUG ===
File: D:\Python_code\RR_Model_1711\sql\uns2.sql
First 200 chars:
 select * from TBL_RR_UNSECURED2
Params: {}


C:\Users\MAFC4709\Python_work\RR_model\src\db.py:84: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(


KeyboardInterrupt: 

In [7]:
matrices_by_mob, parent_fallback = compute_transition_by_mob(df)


⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=500-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=550-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=600-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650+)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=OTHERS)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=A)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 

In [8]:
forecast_results = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=24
)


In [11]:
sale_plan_df = pd.read_excel("C:/Users/User/RR_model/sale_plan_5yF2.xlsx", sheet_name="SALE_PLAN")
sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=24,
    start_state=BUCKETS_CANON[0],
)


In [29]:
import importlib
import src.rollrate.calibration as cal
importlib.reload(cal)


<module 'src.rollrate.calibration' from 'C:\\Users\\User\\RR_model\\src\\rollrate\\calibration.py'>

In [6]:
from src.rollrate.calibration import (
    compute_k_per_product_anchor,
    apply_k_to_lifecycle,
    apply_k_to_sale_plan
)

# 1) Tính hệ số k
k_dict = compute_k_per_product_anchor(
    df_lifecycle=df_lifecycle_actual,      # actual + forecast từ Markov trước calibrate
    forecast_long_df=df_forecast_raw,      # forecast long-format THUẦN Markov
)

print("k per product:", k_dict)

# 2) Apply vào lifecycle forecast
df_lifecycle_calib = apply_k_to_lifecycle(df_lifecycle_actual, k_dict)

# 3) Apply vào sale plan
df_plan_calib = apply_k_to_sale_plan(sale_plan_fc, k_dict)


NameError: name 'df_lifecycle_actual' is not defined

In [53]:
print(k_product)

{'CDLPIL': 0.9997171962709985, 'SALPIL': 0.9411032413359577, 'SPLPIL': 0.9999999999999998, 'TOPUP': 0.8523953431486855, 'TWLPIL': 0.7, 'XSELL': 0.9251537079687773}


In [54]:
forecast_results_cal = apply_calibration_to_forecast(
    forecast_results,
    k_product,
)


In [55]:
sale_plan_fc_cal = apply_calibration_to_sale_plan(
    sale_plan_fc,
    k_product,
)


In [56]:
actual_results = get_actual_all_vintages_amount(df)

# ghép actual + forecast đã calibration
lifecycle = combine_all_lifecycle_amount(actual_results, forecast_results_cal)

# long DF
df_lifecycle = lifecycle_to_long_df_amount(lifecycle)


In [57]:
df_lifecycle = lifecycle_to_long_df_amount(lifecycle)

# Bước quan trọng: tạo DEL30/60/90 + PCT
df_del = add_del_metrics(df_lifecycle, df)

# Seasonality phải chạy trên df_del
seasonality = build_seasonality(df_del)


In [58]:
df_lifecycle_adj = apply_seasonality_to_lifecycle(df_lifecycle, seasonality)


In [59]:
df_plan_fc_adj = apply_seasonality_to_sale_plan(sale_plan_fc_cal, seasonality)


In [60]:
df_plan_fc_adj = apply_seasonality_to_sale_plan(sale_plan_fc_cal, seasonality)


In [61]:
df_del = add_del_metrics(df_lifecycle_adj, df)


In [62]:
df_del = tag_forecast_rows_amount(df_del, df)


In [63]:
df_del_prod = aggregate_to_product(df_del)


C:\Users\User\RR_model\src\rollrate\lifecycle.py:371: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [64]:
df_portfolio = aggregate_products_to_portfolio(df_del_prod)


C:\Users\User\RR_model\src\rollrate\lifecycle.py:437: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


In [65]:
actual_info_prod = (
    df.groupby(["PRODUCT_TYPE", CFG["orig_date"]])[CFG["mob"]].max()
    .reset_index()
    .set_index(["PRODUCT_TYPE", CFG["orig_date"]])[CFG["mob"]]
    .to_dict()
)

actual_info_all = extend_actual_info_with_portfolio(actual_info_prod)


In [66]:
export_lifecycle_all_products_one_file(
    df_del_prod=df_del_prod,
    actual_info=actual_info_all,
    filename="lifecycle_report.xlsx"
)


✔ Export lifecycle multi-product thành công → lifecycle_report.xlsx


## draf

In [7]:
from src.rollrate.pipeline_lifecycle import run_full_pipeline


In [11]:
import pandas as pd

# # 1. Load raw historical data (đầu vào làm markov)
# df_raw = pd.read_parquet("historical_data.parquet")

# # 2. Load sale plan
# sale_plan_df = pd.read_excel("sale_plan.xlsx")
# sale_plan_df["VINTAGE_DATE"] = pd.to_datetime(sale_plan_df["VINTAGE_DATE"])
df_raw = df
sale_plan_df = pd.read_excel("C:/Users/MAFC4709/Python_work/RR_model/sale_plan_5yF2.xlsx", sheet_name="SALE_PLAN")
# 3. RUN FULL PIPELINE
results = run_full_pipeline(
    df_raw=df_raw,
    sale_plan_df=sale_plan_df,
    max_mob=29,
    sale_plan_mob=24,
    export_filename="Lifecycle_Full_Report.xlsx"
)

df_lifecycle_final = results["df_lifecycle"]
df_sale_plan_final = results["df_plan_fc"]
df_report = results["df_final"]
k_values = results["k_dict"]
season = results["seasonality"]



1) COMPUTE TRANSITION
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=500-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=550-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=600-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650+)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=OTHERS)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=A)
⚠️ Có 4 hàng có tổng weight = 0: 

KeyError: 'DISBURSAL_DATE'

In [12]:
from src.rollrate.transition import compute_transition_by_mob

matrices_by_mob, parent_fallback = compute_transition_by_mob(df_raw)


⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=500-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=550-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=600-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650+)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=OTHERS)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=A)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 

In [15]:
from src.rollrate.forecast import forecast_all_vintages

forecast_results = forecast_all_vintages(
    df_raw=df_raw,
    matrices_by_mob=matrices_by_mob,
    max_mob=29,
    enable_macro=False
)


In [16]:
from src.rollrate.forecast_plan import forecast_sale_plan_by_mob
from src.config import BUCKETS_CANON

states = list(dict.fromkeys(list(BUCKETS_CANON) + ["PREPAY", "WRITEOFF"]))

sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=24,
    start_state=BUCKETS_CANON[0],
    states=states
)


In [ ]:
def add_del_metrics_simple(df_plan):
    """
    DEL metrics cho SALE PLAN (không có loan-level DISB).
    """
    df = df_plan.copy()

    bucket_30p = ["DPD30+", "DPD60+", "DPD90+", "DPD120+", "DPD180+", "WRITEOFF"]
    bucket_60p = ["DPD60+", "DPD90+", "DPD120+", "DPD180+", "WRITEOFF"]
    bucket_90p = ["DPD90+", "DPD120+", "DPD180+", "WRITEOFF"]

    # Amount
    df["DEL30_AMT"] = df[bucket_30p].sum(axis=1)
    df["DEL60_AMT"] = df[bucket_60p].sum(axis=1)
    df["DEL90_AMT"] = df[bucket_90p].sum(axis=1)

    # Lấy tổng EAD tại MOB 0
    df["EAD0"] = df.groupby(
        ["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"]
    )["DPD0"].transform("max")

    # Percent
    df["DEL30_PCT"] = df["DEL30_AMT"] / df["EAD0"]
    df["DEL60_PCT"] = df["DEL60_AMT"] / df["EAD0"]
    df["DEL90_PCT"] = df["DEL90_AMT"] / df["EAD0"]

    return df


In [17]:
from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    combine_all_lifecycle_amount,
    lifecycle_to_long_df_amount
)

actual_results = get_actual_all_vintages_amount(df_raw)
lifecycle_dict = combine_all_lifecycle_amount(actual_results, forecast_results)

df_lifecycle = lifecycle_to_long_df_amount(lifecycle_dict)


In [18]:
from src.rollrate.lifecycle import add_del_metrics

df_lifecycle = add_del_metrics(df_lifecycle, df_raw)


In [19]:
from src.rollrate.calibration import compute_k_per_product_anchor, apply_k_to_lifecycle, apply_k_to_sale_plan

k_dict = compute_k_per_product_anchor(
    df_lifecycle=df_lifecycle,
    forecast_long_df=df_lifecycle
)

df_lifecycle = apply_k_to_lifecycle(df_lifecycle, k_dict)
sale_plan_fc = apply_k_to_sale_plan(sale_plan_fc, k_dict)

In [20]:
from src.rollrate.seasonality import (
    build_seasonality,
    apply_seasonality_to_lifecycle,
    apply_seasonality_to_sale_plan
)

seasonality = build_seasonality(df_lifecycle)

df_lifecycle = apply_seasonality_to_lifecycle(df_lifecycle, seasonality)
sale_plan_fc = apply_seasonality_to_sale_plan(sale_plan_fc, seasonality)


##### new try


In [25]:
%load_ext autoreload
%autoreload 1
%aimport src.rollrate.calibration


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import importlib
import src.rollrate.calibration as calibration
calibration = importlib.reload(calibration)

from src.rollrate.calibration import (
    compute_k_per_product_anchor,
    apply_k_to_lifecycle,
    apply_k_to_sale_plan,
)


In [35]:
import pandas as pd
import numpy as np

from src.rollrate.transition import compute_transition_by_mob
from src.rollrate.forecast import forecast_all_vintages
from src.rollrate.forecast_plan import forecast_sale_plan_by_mob

from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    combine_all_lifecycle_amount,
    lifecycle_to_long_df_amount,
    add_del_metrics,
    tag_forecast_rows_amount,
)

from src.rollrate.calibration import (
    compute_k_per_product_anchor,
    apply_k_to_forecast_long_df,
    apply_k_to_sale_plan,
)

from src.rollrate.seasonality import (
    build_seasonality,
    apply_seasonality_to_lifecycle,
    apply_seasonality_to_sale_plan,
)

from src.config import BUCKETS_CANON, CFG


In [36]:
matrices_by_mob, parent_fallback = compute_transition_by_mob(df)
print("✔ Transition matrices ready")


⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=500-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=550-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=600-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650+)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=OTHERS)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=A)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 

In [37]:
forecast_results = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29,
    enable_macro=False,
)
print("✔ Forecast disbursed portfolio done")


✔ Forecast disbursed portfolio done


In [38]:
actual_results = get_actual_all_vintages_amount(df)

lifecycle = combine_all_lifecycle_amount(actual_results, forecast_results)

df_lifecycle = lifecycle_to_long_df_amount(lifecycle)
print(df_lifecycle.head())


  PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB        DPD0      DPD1+     DPD30+  \
0       CDLPIL       500-   2023-01-01    0  493.374711   3.613740   0.000000   
1       CDLPIL       500-   2023-01-01    1  418.023702  21.318179   0.000000   
2       CDLPIL       500-   2023-01-01    2  308.596853  53.241728  18.335200   
3       CDLPIL       500-   2023-01-01    3  265.708119   5.770333  28.830319   
4       CDLPIL       500-   2023-01-01    4  180.169614  28.523850   5.770333   

      DPD60+     DPD90+  DPD120+  DPD180+  PREPAY  WRITEOFF  SOLDOUT  
0   0.000000   0.000000      0.0      0.0     0.0       0.0      0.0  
1   0.000000   0.000000      0.0      0.0     0.0       0.0      0.0  
2   0.000000   0.000000      0.0      0.0     0.0       0.0      0.0  
3  13.005200   0.000000      0.0      0.0     0.0       0.0      0.0  
4  20.863773  17.520001      0.0      0.0     0.0       0.0      0.0  


In [39]:
df_lifecycle = tag_forecast_rows_amount(df_lifecycle, df)
print("✔ Tagged forecast rows")


✔ Tagged forecast rows


In [40]:
df_lifecycle = add_del_metrics(df_lifecycle, df)
print(df_lifecycle.head())


  PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB        DPD0      DPD1+     DPD30+  \
0       CDLPIL       500-   2023-01-01    0  493.374711   3.613740   0.000000   
1       CDLPIL       500-   2023-01-01    1  418.023702  21.318179   0.000000   
2       CDLPIL       500-   2023-01-01    2  308.596853  53.241728  18.335200   
3       CDLPIL       500-   2023-01-01    3  265.708119   5.770333  28.830319   
4       CDLPIL       500-   2023-01-01    4  180.169614  28.523850   5.770333   

      DPD60+     DPD90+  DPD120+  ...  SOLDOUT  ACTUAL_MAX_MOB  IS_FORECAST  \
0   0.000000   0.000000      0.0  ...      0.0              24            0   
1   0.000000   0.000000      0.0  ...      0.0              24            0   
2   0.000000   0.000000      0.0  ...      0.0              24            0   
3  13.005200   0.000000      0.0  ...      0.0              24            0   
4  20.863773  17.520001      0.0  ...      0.0              24            0   

   DISB_TOTAL  DEL30_AMT  DEL60_AMT  D

In [41]:
sale_plan_df = pd.read_excel("sale_plan.xlsx")
sale_plan_df["VINTAGE_DATE"] = pd.to_datetime(sale_plan_df["VINTAGE_DATE"])

# Chuẩn hóa SCORE
def normalize_score(x):
    if pd.isna(x):
        return "ALL"
    s = str(x).strip().upper()
    return "ALL" if s in ["", "NA", "NAN", "NULL"] else s

sale_plan_df["RISK_SCORE"] = sale_plan_df["RISK_SCORE"].apply(normalize_score)

sale_plan_df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'sale_plan.xlsx'

In [ ]:
states = list(dict.fromkeys(list(BUCKETS_CANON) + ["WRITEOFF", "PREPAY"]))

sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=24,
    start_state=BUCKETS_CANON[0],
    states=states,
)

print("✔ Forecast SALE PLAN done")
sale_plan_fc.head()


In [ ]:
seasonality = build_seasonality(df_lifecycle)
print("✔ Seasonality ready")


In [ ]:
df_lifecycle = apply_seasonality_to_lifecycle(df_lifecycle, seasonality)
sale_plan_fc = apply_seasonality_to_sale_plan(sale_plan_fc, seasonality)

print("✔ Applied seasonality")


In [ ]:
# Gán mapping anchor MOB
anchor_map = {
    "CDLPIL": 12,
    "TWLPIL": 12,
    "SPLPIL": 12,
    # default cho các product khác
}

k_dict = compute_k_per_product_anchor(
    df_raw=df,
    forecast_results=forecast_results,
    anchor_map=anchor_map,
)

print(k_dict)


In [ ]:
df_lifecycle = apply_k_to_forecast(df_lifecycle, k_dict)
sale_plan_fc = apply_k_to_sale_plan(sale_plan_fc, k_dict)


In [ ]:
from src.rollrate.export import export_lifecycle_all_products_one_file

# Xây actual_info cho export
actual_info = df_lifecycle.groupby(["PRODUCT_TYPE","VINTAGE_DATE"])["ACTUAL_MAX_MOB"].max().to_dict()

export_lifecycle_all_products_one_file(
    df_del_prod=df_lifecycle,
    actual_info=actual_info,
    filename="Lifecycle_Report.xlsx"
)
